In [5]:
import numpy as np
import os
import tensorflow as tf
import cv2
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd

In [17]:
def model_inputs(video_size):

    #[batch_size, ?, ?, ?]
    #inputs = tf.placeholder(dtype = tf.float32, shape = [None, image_size[0], image_size[1], 3], name= 'videos')
    inputs = tf.placeholder(dtype = tf.float32, shape = [batch_size, video_size[0], video_size[1], video_size[2], 1], name= 'videos')
    targets = tf.placeholder(dtype= tf.int32, shape = [None], name='targets')
    dropout_rate = tf.placeholder(dtype=tf.float32, name='dropout_rate')

    return inputs, targets, dropout_rate

In [13]:
temp_words_list = ['ABSOLUTELY', 'BUDGET', 'EVERYONE', 'HOUSE', 'MILITARY', 'PUBLIC', 'RESULT', 'SIGNIFICANT', 'WEATHER']

In [3]:

video_array = np.load('/home/taylorpap/Bootcamp/CroppedLRW/ABOUT/test/ABOUT_00001.npz')['data']
print(video_array[0][0])
print(video_array.shape)

[[124 134 180]
 [124 136 182]
 [125 137 183]
 [123 135 183]
 [120 133 180]
 [115 131 178]
 [114 130 177]
 [115 131 179]
 [114 130 178]
 [111 127 175]
 [107 123 171]
 [105 119 168]
 [102 116 165]
 [100 114 163]
 [ 99 113 163]
 [ 97 109 161]
 [ 96 107 160]
 [ 97 108 161]
 [ 97 108 162]
 [ 99 110 164]
 [100 111 165]
 [100 110 165]
 [100 110 165]
 [ 98 108 163]
 [ 96 106 162]
 [ 94 105 159]
 [ 97 108 162]
 [ 98 109 163]
 [101 111 165]
 [101 112 164]
 [100 111 163]
 [101 108 163]
 [103 108 165]
 [ 99 105 158]
 [ 90  97 148]
 [ 78  86 134]
 [ 72  80 126]
 [ 85  94 139]
 [107 116 160]
 [112 121 166]
 [109 117 164]
 [111 118 167]
 [115 122 173]
 [119 126 176]
 [119 127 177]
 [119 126 176]
 [118 125 175]
 [117 122 175]
 [116 118 174]
 [114 116 171]
 [110 113 166]
 [106 109 160]
 [102 106 154]
 [ 98 102 149]
 [ 95 100 145]
 [ 94 100 144]
 [ 96 102 144]
 [ 90  96 138]
 [ 80  87 129]
 [ 74  80 123]
 [ 78  84 127]
 [ 80  86 129]
 [ 86  92 136]
 [ 91  96 141]
 [ 93  99 145]
 [ 94 100 146]
 [ 95 102 

In [4]:
video_array = np.load('/media/taylorpap/1TBM2/DatasetML/Lipreading_using_Temporal_Convolutional_Networks-master/datasets/visual_data/ABOUT/test/ABOUT_00001.npz')['data']
print(video_array[0][0])
print(video_array.shape)

[143 146 148 147 144 142 139 136 132 128 125 121 121 121 120 117 118 120
 121 122 122 122 120 113 112 112 117 117 121 120 119 120 121 119 120 102
  88 107 129 130 129 133 139 143 143 139 137 135 133 129 125 121 116 113
 113 114 102  91  90  97 104 110 112 111 113 114 115 115 114 113 111 119
 122 121 122 118 117 119 123 122 123 124 126 126 125 123 122 122 122 122
 122 122 123 124 124 129]
(29, 96, 96)


In [8]:
video_array = np.load('/home/taylorpap/Bootcamp/CroppedLRW/ABSOLUTELY/test/ABSOLUTELY_00006.npz')['data']
print(video_array[1][1])
print(video_array.shape)

[[  0  11  18]
 [  0  10  17]
 [  0  16  24]
 [  5  25  33]
 [  7  31  42]
 [  8  33  45]
 [  6  36  53]
 [ 13  44  61]
 [ 26  59  75]
 [ 39  73  92]
 [ 48  83 102]
 [ 51  88 110]
 [ 55  93 115]
 [ 59  99 122]
 [ 61 102 125]
 [ 60 101 127]
 [ 56  98 124]
 [ 53  96 123]
 [ 52  94 123]
 [ 50  92 122]
 [ 55  95 125]
 [ 60  99 130]
 [ 68 110 142]
 [ 81 123 155]
 [ 90 132 164]
 [ 91 133 166]
 [ 88 131 165]
 [ 86 129 163]
 [ 85 128 162]
 [ 86 127 163]
 [ 87 127 164]
 [ 89 129 166]
 [ 98 138 175]
 [110 148 192]
 [122 160 206]
 [126 165 216]
 [129 169 221]
 [131 172 225]
 [133 175 230]
 [133 176 231]
 [131 174 229]
 [128 171 226]
 [126 169 224]
 [121 164 218]
 [116 157 210]
 [113 153 205]
 [112 151 202]
 [112 154 201]
 [110 154 199]
 [105 150 193]
 [103 148 191]
 [100 145 188]
 [ 99 144 187]
 [ 96 141 184]
 [ 96 141 184]
 [ 96 141 184]
 [ 95 140 183]
 [ 97 142 185]
 [101 146 189]
 [104 149 192]
 [105 150 193]
 [104 149 192]
 [104 149 190]
 [103 149 190]
 [101 148 189]
 [ 99 144 184]
 [ 94 137 

In [7]:
def sparse_accuracy(true_labels, predicted_labels):

    assert len(true_labels) == len(predicted_labels)

    correct = 0

    for i in range(len(true_labels)):

        if np.argmax(predicted_labels[i]) == true_labels[i]:
            correct +=1

    return correct / len(true_labels)

In [8]:
def conv_block(inputs,
               number_of_filters,
               kernel_size,
               strides=(1,1),
               padding='SAME',
               activation= tf.nn.relu,
               max_pool=True,
               batch_norm=True):

    conv_features = layer = tf.layers.conv2d(inputs=inputs,
                                             filters=number_of_filters,
                                             kernel_size=kernel_size,
                                             strides=strides,
                                             padding=padding,
                                             activation=activation)

    if max_pool:
        layer = tf.layers.max_pooling2d(layer,
                                        pool_size=(2, 2),
                                        strides=(2, 2),
                                        padding='SAME')

    if batch_norm:
        layer = tf.layers.batch_normalization(layer)

    return layer, conv_features

In [9]:
def conv3d_block(inputs, number_of_filters, kernel_size, strides = None, padding='SAME', activation= tf.nn.relu, max_pool=True):

    conv_features = layer = tf.layers.Conv3D(inputs=inputs,
                                                filters=number_of_filters,
                                                kernel_size=kernel_size,
                                                strides=strides,
                                                padding=padding,
                                                activation=activation)

    if max_pool:
        layer = tf.layers.MaxPool3D(layer, pool_size = (3,3,3), strides = None, padding = 'SAME')

    return layer, conv_features

In [10]:
def dense_block(inputs,
                units,
                activation=tf.nn.relu,
                dropout_rate=None,
                batch_norm=True):


    dense_features = layer = tf.layers.dense(inputs,
                                             units=units,
                                             activation=activation)

    if dropout_rate is not None:
        layer = tf.layers.dropout(layer, rate=dropout_rate)

    if batch_norm:
        layer = tf.layers.batch_normalization(layer)

    return layer, dense_features

In [11]:
def opt_loss(logits,
             targets,
             learning_rate):

    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=targets, logits=logits))

    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

    return loss, optimizer

In [15]:
class LipreadVideoModel(object):
    def __init__(self, learning_rate, video_dims, number_of_classes):

        #tf.reset_default_graph()

        self.inputs, self.targets, self.dropout_rate = model_inputs(video_dims)

        preprocessed_arrays = self.inputs

        #Conv3Dblock 1
        conv3d_block_1, self.conv_1_features = conv3d_block(inputs=preprocessed_arrays,
                                                            number_of_filters=32,
                                                            kernel_size=(3,3,3),
                                                            padding='SAME',
                                                            activation= tf.nn.relu,
                                                            max_pool=False)

        #Conv3Dblock 2
        conv3d_block_2, self.conv_2_features = conv3d_block(inputs=conv3d_block_1,
                                                            number_of_filters=64,
                                                            kernel_size=(3,3,3),
                                                            padding='SAME',
                                                            activation= tf.nn.relu,
                                                            max_pool=True)

        #Conv3Dblock 3
        conv3d_block_3, self.conv_3_features = conv3d_block(inputs=conv3d_block_2,
                                                            number_of_filters=64,
                                                            kernel_size=(3,3,3),
                                                            padding='SAME',
                                                            activation= tf.nn.relu,
                                                            max_pool=False)

        #Conv3Dblock 4
        conv3d_block_4, self.conv_4_features = conv3d_block(inputs=conv3d_block_3,
                                                            number_of_filters=128,
                                                            kernel_size=(5,5,5),
                                                            padding='SAME',
                                                            activation= tf.nn.relu,
                                                            max_pool=True)

        #flatten
        flat_layer = tf.keras.layers.Flatten(conv3d_block_4)

        dense_block_1, dense_1_features = dense_block(flat_layer,
                                                      units=512,
                                                      activation=tf.nn.relu,
                                                      dropout_rate=self.dropout_rate,
                                                      batch_norm=True)

        dense_block_2, dense_2_features = dense_block(dense_block_1,
                                                      units=256,
                                                      activation=tf.nn.relu,
                                                      dropout_rate=self.dropout_rate,
                                                      batch_norm=True)

        dense_block_3, dense_3_features = dense_block(dense_block_2,
                                                      units=128,
                                                      activation=tf.nn.relu,
                                                      dropout_rate=self.dropout_rate,
                                                      batch_norm=True)

        logits = tf.keras.layers.dense(inputs=dense_block_3,
                                       units=number_of_classes,
                                       activation=None)

        self.predictions = tf.nn.softmax(logits)

        self.loss, self.opt = opt_loss(logits=logits,
                                       targets=self.targets,
                                       learning_rate=learning_rate)

In [18]:
first_model = LipreadVideoModel(0.001, (29, 88, 88), len(temp_words_list))

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [16]:

def train(model,
          epochs,
          drop_rate,
          batch_size,
          data,
          save_dir,
          saver_delta=0.15):

    '''
    The core training function, use this function to train a model.

    :param model: CNN model
    :param epochs: integer, number of epochs
    :param drop_rate: float, dropout_rate
    :param batch_size: integer, number of samples to put through the model at once
    :param data: tuple, train-test data Example(X_train, y_train, X_test, y_test)
    :param save_dir: string, path to a folder where model checkpoints will be saved
    :param saver_delta: float, used to prevent overfitted model to be saved
    '''

    X_train, y_train, X_test, y_test = data

    #start session
    session = tf.Session()
    session.run(tf.global_variables_initializer())

    #define saver
    saver = tf.train.Saver()

    best_test_accuracy = 0.0
    #start training loop
    for epoch in range(epochs):

        train_accuracy = []
        train_loss = []

        for ii in tqdm(range(len(X_train) // batch_size)):
            start_id = ii*batch_size
            end_id = start_id + batch_size

            X_batch = X_train[start_id:end_id]
            y_batch = y_train[start_id:end_id]

            feed_dict = {model.inputs:X_batch,
                         model.targets:y_batch,
                         model.dropout_rate:drop_rate}

            _, t_loss, preds_t = session.run([model.optimizer, model.loss, model.predictions], feed_dict=feed_dict)

            train_accuracy.append(sparse_accuracy(y_batch, preds_t))
            train_loss.append(t_loss)

        print("Epoch: {}/{}".format(epoch, epochs),
              " | Training accuracy: {}".format(np.mean(train_accuracy)),
              " | Training loss: {}".format(np.mean(train_loss)) )

        test_accuracy = []

        for ii in tqdm(range(len(X_test) // batch_size)):
            start_id = ii*batch_size
            end_id = start_id + batch_size

            X_batch = X_test[start_id:end_id]
            y_batch = y_test[start_id:end_id]

            feed_dict = {model.inputs:X_batch,
                         model.dropout_rate:0.0}

            preds_test = session.run(model.predictions, feed_dict=feed_dict)
            test_accuracy.append(sparse_accuracy(y_batch, preds_test))

        print("Test accuracy: {}".format(np.mean(test_accuracy)))

        #saving the model
        if np.mean(train_accuracy) > np.mean(test_accuracy): #to prevent underfitting
            if np.abs(np.mean(train_accuracy) - np.mean(test_accuracy)) <= saver_delta: #to prevent overfit
                if np.mean(test_accuracy) >= best_test_accuracy:
                    best_test_accuracy = np.mean(test_accuracy)
                    saver.save(session, "{}/model_epoch_{}.ckpt".format(save_dir, epoch))

    session.close()